In [1]:
from triple_stream_data_utils import plot_scatter_distribution, create_heatmap_histogram_from_lists
from tripleStreamsDataloaderUtils import load_compiled_dataset_pkl_bz2

Could not import fluidsynth. AUDIO rendering will not work.
/Users/bezha/anaconda3/envs/TripleStreams/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Make sure you've already ran python CompileStreamsAndFeatures.py
import os

root = "data/triple_streams/split_2bars/compiled_data/"
all_pkl_bz2_files = [f for f in os.listdir(root) if f.endswith('.pkl.bz2')]

# Available Datasets
available_datasets_path = {
    all_pkl_bz2_file.split(".pkl.bz2")[0]: os.path.join(root, all_pkl_bz2_file) for all_pkl_bz2_file in all_pkl_bz2_files
}
# dict_keys(['input_hvos', 'output_hvos', 'flat_out_hvos', 'sample_id', 'collection', 'metadata', 'qpm', 'OF_Input Hamming', 'OF_Input Hamming Accent', 'OS1_OF Jaccard', 'OS2_OF Jaccard', 'OS3_OF Jaccard', 'pearson', 'cosine', 'euclidean_similarity', 'mae_similarity', 'dtw_similarity'])

available_datasets_path,

({'lmd_drum_guitar_percussion_piano': 'data/triple_streams/split_2bars/compiled_data/lmd_drum_guitar_percussion_piano.pkl.bz2',
  'lmd_drum_guitar_percussive_piano': 'data/triple_streams/split_2bars/compiled_data/lmd_drum_guitar_percussive_piano.pkl.bz2',
  'lmd_bass_brass_guitar_piano': 'data/triple_streams/split_2bars/compiled_data/lmd_bass_brass_guitar_piano.pkl.bz2',
  'groove_midi_crash_hhclosed_hhopen_ride': 'data/triple_streams/split_2bars/compiled_data/groove_midi_crash_hhclosed_hhopen_ride.pkl.bz2',
  'lmd_bass_percussion_percussive_piano': 'data/triple_streams/split_2bars/compiled_data/lmd_bass_percussion_percussive_piano.pkl.bz2',
  'lmd_bass_drum_guitar_piano': 'data/triple_streams/split_2bars/compiled_data/lmd_bass_drum_guitar_piano.pkl.bz2',
  'groove_midi_hh_kick_snare_toms': 'data/triple_streams/split_2bars/compiled_data/groove_midi_hh_kick_snare_toms.pkl.bz2',
  'elbg_both_flattened_left_right': 'data/triple_streams/split_2bars/compiled_data/elbg_both_flattened_left_ri

In [3]:
from make_triple_stream_plots import build_plots

feat1 = "OF_Input Hamming"
feat2 = "OF_Input Hamming Accent"
# dataset_tags = ["lmd_bass_brass_guitar_percussive", "elbg_both_flattened_left_right"]
dataset_tags = [f.split("/")[-1] for f in os.listdir(root) if f.endswith('.pkl.bz2')];

html_path = f"data/triple_streams/split_2bars/plots/<{feat1}>vs<{feat2}>.html"
os.makedirs(os.path.dirname(html_path), exist_ok=True)

# This will generate the HTML and return its path
output_path = build_plots(
    root="data/triple_streams/split_2bars/compiled_data/",
    dataset_tags=["lmd_bass_brass_guitar_percussive", "elbg_both_flattened_left_right"],
    feature1=feat1,
    feature2=feat2,
    n_bins_feat1=None,
    n_bins_feat2=None,
    scatter_max_points=10000000,
    output_html_path=f"data/triple_streams/split_2bars/plots/<{feat1}>vs<{feat2}>.html"
)

print(f"HTML saved at: {output_path}")

Loading datasets...


Datasets: 100%|██████████| 2/2 [00:00<00:00,  2.83it/s]


Preparing heatmap caches...


Heatmaps: 100%|██████████| 2/2 [00:00<00:00, 315.05it/s]


Preparing scatter data...


Scatter: 100%|██████████| 2/2 [00:00<00:00, 99.83it/s]


HTML saved at: /Users/bezha/PycharmProjects/TripleStreams/data/triple_streams/split_2bars/plots/<OF_Input Hamming>vs<OF_Input Hamming Accent>.html


In [4]:
# dataset_tags = ["lmd_bass_brass_guitar_percussive", "elbg_both_flattened_left_right"]   # "lmd" is not included here
# loaded_datasets = {}
#
# for dataset_tag in dataset_tags:
#     # load .pkl.bz2 file
#     if dataset_tag in available_datasets_path:
#         d_dict = load_compiled_dataset_pkl_bz2(available_datasets_path[dataset_tag])
#         for key, val in d_dict.items():
#             if key not in loaded_datasets:
#                 loaded_datasets[key] = []
#             loaded_datasets[key].extend(val)
# print(loaded_datasets.keys())

In [5]:
# sample_ix = 100 # for each sample 24 variations
# input_hvos = loaded_datasets["input_hvos"][sample_ix*24: ((sample_ix+1)*24)]
# output_hvos = loaded_datasets["output_hvos"][sample_ix*24: ((sample_ix+1)*24)]
# metadatas = loaded_datasets["metadata"][sample_ix*24: ((sample_ix+1)*24)]
# print(len(input_hvos), len(output_hvos), len(metadatas))
# print(loaded_datasets["OF_Input Hamming"][sample_ix*24: ((sample_ix+1)*24)])
# print(loaded_datasets["OF_Input Hamming Accent"][sample_ix*24: ((sample_ix+1)*24)])
# print(loaded_datasets["OS1_OF Jaccard"][sample_ix*24: ((sample_ix+1)*24)])
# print(loaded_datasets["OS2_OF Jaccard"][sample_ix*24: ((sample_ix+1)*24)])
# print(loaded_datasets["OS3_OF Jaccard"][sample_ix*24: ((sample_ix+1)*24)])
# create_multitab_from_HVO_Sequences(compile_into_list_of_hvo_seqs(input_hvos, output_hvos, metadatas, qpms=None))